In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# points in polygons
def get_pip(points, polygon):
    id_list = list(polygon.GEOID)
    df = pd.DataFrame().reindex_like(points).dropna()
    for ID in id_list:
        pol = (polygon.loc[polygon.GEOID==ID])
        pol.reset_index(drop = True, inplace = True)
        pip_mask = points.within(pol.loc[0, 'geometry'])
        pip_data = points.loc[pip_mask].copy()
        pip_data['GEOID']= ID
        df = df.append(pip_data)
    df.reset_index(inplace=True, drop=True)
    df = df.drop(columns='geometry')
    return df

In [3]:
path = r'T:\DCProjects\Modeling\AADBT\input\shp'

In [4]:
bike_counts_loc = gpd.read_file(path + '\\BikeCountsLocations.shp')

In [5]:
b_dens = gpd.read_file(path +'\\building_density_census_tract.shp')

In [6]:
pnt_in_ply = get_pip(points = bike_counts_loc, polygon = b_dens)

In [12]:
ndata = pnt_in_ply.merge(b_dens[['GEOID', 'b_dens']], on='GEOID')

In [15]:
outpath = r'T:\DCProjects\Modeling\AADBT\input'

In [17]:
data = pd.read_csv(outpath + '\\Daily_Bike_Counts_With_AllVarData.csv')

In [19]:
ndata.rename(columns={"b_dens": "BuilDens"}, inplace=True)

In [20]:
df = data.merge(ndata[['Location', 'BuilDens']], on='Location')

In [22]:
df.to_csv(outpath + '\\Daily_Bike_Counts_With_AllVarData.csv', index=False)